In [2]:
import os
import sys
import argparse
import torch
import random
import torch.nn.functional as F

import numpy as np
import pandas as pd

In [3]:
tripod_train=pd.read_csv('~/lab/TRIPOD/Synopses_and_annotations/TRIPOD_synopses_train.csv')
tripod_test=pd.read_csv('~/lab/TRIPOD/Synopses_and_annotations/TRIPOD_synopses_test.csv')
print(tripod_train.head)

<bound method NDFrame.head of                 movie_name                                       synopsis_raw  \
0         Beloved (film)_1  Set shortly after the Civil War, the film revo...   
1         Beloved (film)_0  Set shortly after the Civil War, the film revo...   
2         Beloved (film)_2  Set shortly after the Civil War, the film revo...   
3            Jaws (film)_0  A girl named Chrissie Watkins leaves a beach p...   
4      Angel Eyes (film)_1  On a wet rainy night in Chicago, police office...   
..                     ...                                                ...   
123           Kalifornia_1  Brian Kessler (David Duchovny) is a graduate s...   
124  Titanic (1997 film)_0  In 1996, treasure hunter Brock Lovett and his ...   
125    Twins (1988 film)_2  Julius Benedict (Arnold Schwarzenegger) and Vi...   
126    Twins (1988 film)_0  Julius Benedict (Arnold Schwarzenegger) and Vi...   
127    Twins (1988 film)_1  Julius Benedict (Arnold Schwarzenegger) and Vi...  

In [4]:
from coref_utils import *

In [5]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
coref_model = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import allennlp_models.tagging

srl_model = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

In [45]:
def get_protagonist(coref_model,story):
    coref_prediction=coref_model.predict(document=story)

    ref_len=[len(ref) for ref in coref_prediction["clusters"]]
    print(max(ref_len))
    sorted=np.argsort(np.array(ref_len))
    print(sorted[-1])

    #Tokenize Story
    coref_tokenizer=coref_model._spacy
    document=coref_tokenizer(story)
    # tokenized = list(tok.text_with_ws for tok in document)
    tokenized = list(tok.text for tok in document)

    most_ref=coref_prediction["clusters"][sorted[-1]][0]
    print(tokenized[most_ref[0]:most_ref[1]+1])

    return " ".join(tokenized[most_ref[0]:most_ref[1]+1])

In [47]:
import spacy
from spacy.lang.en import English
from nltk import tokenize
#Regex
import re
def collect_protagonist_actions(story_df,coref_model,srl_model):
    #Ignoring Verbs
    skip_verbs=["is", "was", "were",
        "are", "be", " ́s", " ́re", " ́ll", "can", "could", "must", "may", "have to", "has to", "had to", "will", "would", "has",        "have", "had", "do", "does", "did"]

    for idx in range(story_df.shape[0]):
        story=story_df.iloc[-3]#[idx]
        story_seg=story["synopsis_segmented"]
        story_split=story_seg.split(" [END_SENT] ")
        story_txt=" ".join([sent[11:] for sent in story_split])

        #Coref Resolve
        story_protagonist=get_protagonist(coref_model,story_txt)

        story_resolved=coref_model.coref_resolved(story_txt)

        resolved_split=tokenize.sent_tokenize(story_resolved)
        
        print("Protagonist:",story_protagonist)
        
        v_preds=[]
        w_preds=[]
        for sent in resolved_split:
            print(sent)
            srl_prediction=srl_model.predict(sent)
            #Single Prediction has Multiple Frames
            #Verb: verb, description, tags
            v_preds.append(srl_prediction["verbs"])
            w_preds.append(srl_prediction["words"])

            #Get Protagonist action
            for i in range(len(srl_prediction["verbs"])):
                frame_args={}
                frame_verb=srl_prediction["verbs"][i]["verb"]
                print("\nverb:",frame_verb)
                # print(srl_prediction["verbs"][i]["description"])
                # print(srl_prediction["verbs"][i]["tags"])
                if frame_verb in skip_verbs:
                    continue
                # args=re.search(r'\[ARG(.*?)\]',srl_prediction["verbs"][i]["description"])
                # args=re.search(r'\[(.*?)\]',srl_prediction["verbs"][i]["description"])
                args=re.findall(r'\[ARG(.*?)\]',srl_prediction["verbs"][i]["description"])
                if args is not None:
                    for arg in args:
                        print(arg)
                        arg_parts=arg.split(":")
                        
                        print(arg_parts)
                        if (story_protagonist in arg_parts[1]):
                            print("Protagonist in",arg_parts[1])
                            arg_parts[1]=arg_parts[1].replace(story_protagonist,"protagonist")
                        # elif (story_protagonist[:-1] in arg_parts[1]):
                        #     print("Protagonist in",arg_parts[1])
                        #     arg_parts[1]=arg_parts[1].replace(story_protagonist[:-1],"protagonist")
                        frame_args[arg_parts[0]]=arg_parts[1][1:]
                # arg_dict=eval("{"+",".join(frame_args)+"}")
                print(frame_args)
                    # print(args)
                    # print(args.group(1))
                # print(re.search(r'\[(.*?)\]',srl_prediction["verbs"][i]["description"]).group(1))
                # print(re.search(r'\[ARG(.*?)\]',srl_prediction["verbs"][i]["description"]).group(1))
            break
        break

    actions=[]

    return actions

In [9]:
diehard=tripod_test.iloc[-2]
diehard_segtext=diehard["synopsis_segmented"]
# print(diehard_segtext)
diehard_split=diehard_segtext.split(" [END_SENT] ")

sample=" ".join([sent[11:] for sent in diehard_split])
print(sample)

Sixty-year-old magnate Robert Miller manages a hedge fund with his daughter Brooke (Brit Marling) and is about to sell it for a handsome profit. However, unbeknownst to his daughter and most of his other employees, he has cooked his company's books in order to cover an investment loss and avoid being arrested for fraud. One night, while driving with his mistress Julie Cote (Laetitia Casta), he begins to doze off and crashes; Julie is killed. An injured Miller leaves the scene and decides to cover up his involvement to prevent the public, his wife Ellen (Susan Sarandon), and the prospective buyer James Mayfield (Graydon Carter) from discovering the truth. Miller calls Jimmy Grant (Nate Parker), a twenty-three-year-old man from Harlem with a criminal record whom he helped get off the street in the past, and whose father had been Miller's driver for many years. After being driven home by Grant, Miller drags his injured body into bed at 4:30 am, arousing suspicion in his wife. The next day

In [10]:
get_protagonist(coref_model,sample)

44
0
['Sixty', '-', 'year', '-', 'old ', 'magnate ', 'Robert ', 'Miller ']


'Sixty - year - old  magnate  Robert  Miller '

In [11]:
characters=get_best_match(coref_model,coref_model.predict(document=sample),sample)

Sixty-year-old magnate Robert Miller
Sixty - year - old magnate Robert Miller
[0, 7]
match: Match(a=0, b=0, size=8)
[13, 13]
[35, 35]
[40, 40]
[44, 44]
[47, 47]
[71, 71]
[80, 80]
[92, 94]
match: Match(a=7, b=2, size=1)
[103, 103]
[110, 110]
[133, 133]
match: Match(a=0, b=0, size=1)
[173, 174]
match: Match(a=0, b=0, size=1)
[187, 187]
match: Match(a=0, b=0, size=1)
[189, 189]
[201, 201]
[208, 208]
[225, 225]
match: Match(a=0, b=0, size=1)
[250, 251]
match: Match(a=0, b=0, size=0)
[269, 269]
match: Match(a=0, b=0, size=1)
[271, 271]
match: Match(a=0, b=0, size=1)
[276, 276]
[289, 289]
match: Match(a=0, b=0, size=1)
[297, 297]
[321, 321]
match: Match(a=0, b=0, size=1)
[330, 330]
[355, 356]
match: Match(a=0, b=0, size=1)
[361, 361]
[369, 369]
[373, 374]
match: Match(a=0, b=1, size=1)
[379, 379]
[389, 389]
[412, 412]
match: Match(a=0, b=0, size=1)
[417, 417]
[419, 419]
[424, 424]
[427, 427]
[430, 430]
[433, 433]
[446, 446]
[454, 454]
[463, 463]
[467, 467]
[481, 481]
a hedge fund
hedge fund


In [12]:
print(characters)

['Miller', 'hedge fund', 'daughter', 'Julie', 'wife', 'Bryer', 'sell', 'crashes']


In [48]:
collect_protagonist_actions(tripod_test,coref_model,srl_model)

41
0
['New', 'York', 'City', 'Police', 'officer', 'John', 'McClane']
Protagonist: New York City Police officer John McClane
On Christmas Eve, New York City Police officer John McClane arrives in Los Angeles to reconcile with New York City Police officer John McClane's estranged wife, Holly Gennero.

verb: arrives
M-TMP: On Christmas Eve
['M-TMP', ' On Christmas Eve']
1: New York City Police officer John McClane
['1', ' New York City Police officer John McClane']
Protagonist in  New York City Police officer John McClane
4: in Los Angeles
['4', ' in Los Angeles']
M-PRP: to reconcile with New York City Police officer John McClane 's estranged wife , Holly Gennero
['M-PRP', " to reconcile with New York City Police officer John McClane 's estranged wife , Holly Gennero"]
Protagonist in  to reconcile with New York City Police officer John McClane 's estranged wife , Holly Gennero
{'M-TMP': 'On Christmas Eve', '1': 'protagonist', '4': 'in Los Angeles', 'M-PRP': "to reconcile with protagonist 

[]